In [ ]:
import cv2
import os

def load_cascade(cascade_path):
    if not os.path.exists(cascade_path):
        print(f"Warning: {cascade_path} not found")
        return None
    return cv2.CascadeClassifier(cascade_path)

face_cascade_path = "C:/Users/opste/Downloads/haarcascade_frontalface_default.xml"
nose_cascade_path = "C:/Users/opste/Downloads/haarcascade_mcs_nose.xml"
mouth_cascade_path = "C:/Users/opste/Downloads/haarcascade_mcs_mouth.xml"

face_cascade = load_cascade(face_cascade_path)
nose_cascade = load_cascade(nose_cascade_path)
mouth_cascade = load_cascade(mouth_cascade_path)

if face_cascade is None or nose_cascade is None or mouth_cascade is None:
    print("Error: Could not load cascades. Exiting.")
    exit()

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

cv2.namedWindow("Face, Nose, and Mouth Detection", cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't receive frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, 'Face', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        noses = nose_cascade.detectMultiScale(roi_gray, 1.1, 5, minSize=(20, 20))
        for (nx, ny, nw, nh) in noses:
            cv2.rectangle(roi_color, (nx, ny), (nx + nw, ny + nh), (255, 0, 0), 2)
            cv2.putText(roi_color, 'Nose', (nx, ny - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            break

        mouths = mouth_cascade.detectMultiScale(roi_gray, 1.1, 5, minSize=(30, 15))
        for (mx, my, mw, mh) in mouths:
            if my > h // 2:
                cv2.rectangle(roi_color, (mx, my), (mx + mw, my + mh), (255, 0, 255), 2)
                cv2.putText(roi_color, 'Mouth', (mx, my - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
                break

    cv2.imshow("Detection of face,nose, mouth", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
